In [ ]:
#!! My second kaggle submission, My computer is getting crazy but that's fun

In [ ]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import mnist
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')


In [ ]:
train.head(15)

In [ ]:

test.head()

In [ ]:
train.describe()

In [ ]:
y_train = train['label']
train = train.drop('label',axis=1)
train.head()

In [ ]:
train.isnull().any().describe()

In [ ]:
test.isnull().any().describe()

In [ ]:
train = train/255.0
test = test/255.0

train = train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
fig = plt.figure(figsize=(20,20))
for i in range(5):
    ax = fig.add_subplot(1, 5, i+1, xticks=[], yticks=[])
    ax.imshow(train[i].reshape(28,28), cmap='gray')
    ax.set_title(str(y_train[i]))

In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes=10)
 
print('One-hot Encoded labels:')
print(y_train[:1])

In [ ]:
random_seed = 2
train, X_val, y_train, Y_val = train_test_split(train, y_train, test_size = 0.1, random_state=random_seed)
print(train.shape, y_train.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))
model.summary()

In [ ]:
# Define Optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:

reduce_lr = ReduceLROnPlateau(monitor='loss',
                              factor=0.3, 
                              verbose=1,
                              patience=2, 
                              min_lr=0.00000001)

hist = model.fit(train, 
                 y_train, 
                 batch_size=128, 
                 epochs=16,
                 validation_data=(X_val, Y_val), 
                 callbacks=[reduce_lr],
                 verbose=1, 
                 shuffle=True)

In [ ]:
Y_test = model.predict_classes(test, verbose=2)

In [ ]:
submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submission['Label'] = Y_test
submission.to_csv('submission.csv',index=False)